In [21]:
import os 
print(os.environ)

# print(f"{os.getenv('EURI_API_KEY')}")

environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\Abhishek Maity\\AppData\\Roaming', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPOSE_CONVERT_WINDOWS_PATHS': 'true', 'COMPUTERNAME': 'ABHISHEKS-ACER-', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'CONDA_BAT': 'D:\\AnacondaFiles\\condabin\\conda.bat', 'CONDA_DEFAULT_ENV': 'base', 'CONDA_EXE': 'D:\\AnacondaFiles\\condabin\\..\\Scripts\\conda.exe', 'CONDA_PREFIX': 'D:\\AnacondaFiles', 'CONDA_PROMPT_MODIFIER': '(base) ', 'CONDA_PYTHON_EXE': 'D:\\AnacondaFiles\\python.exe', 'CONDA_SHLVL': '1', 'CUDA_PATH_V10_0': 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.0', 'CUDA_PATH_V10_1': 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'EFC_9924_1262719628': '1', 'EFC_9924_1592913036': '1', 'EFC_9924_22

### In Pinecone databases are known as indexes

In [34]:
# !pip install pinecone
# !pip install dotenv

In [97]:
import os
from dotenv import load_dotenv
load_dotenv()

True

#### initializing the pinecone db

In [35]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

In [36]:
pc

### Upsert data - creating index

In [42]:
index_name = "sudhanshu"

# if not pc.has_index(index_name):
#     pc.create_index_for_model(
#         name=index_name,
#         cloud="aws",
#         region="us-east-1",
#         embed={
#             "model":"llama-text-embed-v2",
#             "field_map":{"text": "chunk_text"}
#         }
#     )

pc.create_index(
    name = index_name,
    dimension=1536,   # replace with your embedding dimensions
    metric="cosine",  # replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )

)

In [43]:
pc.list_indexes()

[
    {
        "name": "sudhanshu",
        "dimension": 1536,
        "metric": "cosine",
        "host": "sudhanshu-rj6ooka.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [40]:
pc.delete_index(index_name)

### Storing data into indexes 
    - First create embeddings of the texts then store it into the vector databases

In [50]:
import requests
import numpy as np

def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": os.getenv('EURI_API_KEY')
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

text = "The food was delicious and the service was excellent."

embeddings_data = generate_embeddings(text)

In [56]:
embeddings_data

array([-0.03244253, -0.00839565, -0.06356495, ..., -0.00225143,
        0.05896459, -0.03152246])

In [58]:
print(f"LENGTH OF EMBEDDINGS = {len(embeddings_data)}")

LENGTH OF EMBEDDINGS = 1536


In [53]:
index_name

'sudhanshu'

#### Inserting data into pinecone index

In [71]:
index = pc.Index(index_name)
index.upsert([("item-id-001", embeddings_data.tolist(), {"name": index_name, "info": "personal_info"})], 
            namespace= "sudh1")

{'upserted_count': 1}

In [72]:
index.upsert([("item-id-002", embeddings_data.tolist(), {"text": "this is sample text", "metadata": "personal_info"})])

{'upserted_count': 1}

In [73]:
index.fetch(ids=["item-id-001"])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'item-id-001': {'id': 'item-id-001',
                             'metadata': {'metadata': 'personal_info',
                                          'text': 'this is sample text'},
                             'values': [-0.0324425288,
                                        -0.00839565415,
                                        -0.0635649487,
                                        -0.0426833257,
                                        0.0470436625,
                                        -0.0038978036,
                                        -0.0037727938,
                                        0.046843648,
                                        -0.0048578782,
                                        -0.038663011,
                                        0.0257420056,
                                        -0.0293422863,
                                        0.00562043767,
                                        -0.0489

In [74]:
index.fetch(ids=["item-id-002"])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'item-id-002': {'id': 'item-id-002',
                             'metadata': {'metadata': 'personal_info',
                                          'text': 'this is sample text'},
                             'values': [-0.0324425288,
                                        -0.00839565415,
                                        -0.0635649487,
                                        -0.0426833257,
                                        0.0470436625,
                                        -0.0038978036,
                                        -0.0037727938,
                                        0.046843648,
                                        -0.0048578782,
                                        -0.038663011,
                                        0.0257420056,
                                        -0.0293422863,
                                        0.00562043767,
                                        -0.0489

### Searching from vector DB

In [66]:
text = "i ate food in dinner"

embeddings_to_search = generate_embeddings(text)
embeddings_to_search

array([-0.00142219, -0.02961568, -0.0201144 , ...,  0.0269624 ,
        0.02158003, -0.01672831])

In [70]:
index.query(
    vector=embeddings_to_search.tolist(),
    top_k=2,
    include_metadata=True
)

{'matches': [{'id': 'item-id-002',
              'metadata': {'metadata': 'personal_info',
                           'text': 'this is sample text'},
              'score': 0.327314377,
              'values': []},
             {'id': 'item-id-001',
              'metadata': {'metadata': 'personal_info',
                           'text': 'this is sample text'},
              'score': 0.327314377,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

# Weaviate DB

In [86]:
# !pip install weaviate-client

In [99]:
# os.getenv("WEAVIATE_API_KEY")

In [103]:
# import os
# import weaviate
# from weaviate.classes.init import Auth

# # Connect to weaviate cloud
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url= "ofoebchmrkqim4m3sodacw.c0.asia-southeast1.gcp.weaviate.cloud",
#     auth_credentials=Auth.api_key(os.getenv("WEAVIATE_API_KEY"))

# )